In [ ]:
import heapq
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# static during simulation can be dynamic on settings (solids rate, converstion etc)
# define static tank properties: name, capacity, HH/H/L/LL_perc
# define initial dynamic tank properies: level_ltrs, level_perc,
# define static rates of flows: make batch, feed emu, output emu, feed filtrate larox, etc
# define static duration of events: Larox timings

  # que of events
'''
# function for defining events
#Events have a start_time, end_time, duration, description, effected tank, rate of fill/drain
# After an event, the tanks are updated for their effected Properties
The end point of the event is simulated
Then a loop along possible triggers to determine if additional events are triggered
Then the current event que needs updating (although we can track where we are in the event given begin, end and duration)
This could fill the events_que and we start at the event que again
'''

# loop over events,
  # determine timestep for this step (shortest timestep of events)
  # update tank levels,
  # determine triggers for new events,
  # define new events by determine dydnamic rates/durations, add to que
  #

  #triggers
# fill feedbatch: if tank is <xx , not in use, and time is between 6-(22-fill_time)
      # feetanks should have a status_Flag
# start EMU: if feed_tank > xx, keep running until slurry > xx or feedtank < xx and other tank is < xx
# swith feedtank: if feed_tank <xx and other tank is > xx
# start Larox: if slurry is > level, Filtrate is < level, trigger 1 cycle. Later add: pause during cycle if slurry < xx or Filtrate is > xx
# start filtrate discharge: triggered by feedbatch?
# drain filtrate?

# Define the tanks with static properties
tanks = pd.DataFrame({
    'Tank': ['FeedTank1', 'FeedTank2', 'SlurryTank', 'FiltrateTank'],
    'MaxCapacity': [6500, 6500, 6500, 6500],  # Capacities in liters
    'HH': [0.9, 0.9, 0.9, 0.9],
    'H': [0.8, 0.8, 0.8, 0.8],
    'L': [0.2, 0.2, 0.2, 0.2],
    'LL': [0.1, 0.1, 0.1, 0.1],
})
#pre-calculate other static properties
tanks['HCapacity'] = tanks['MaxCapacity'] * tanks['H']
tanks['LCapacity'] = tanks['MaxCapacity'] * tanks['L']

# Define the rates in liters per hour
rates = {
    'Feed_Tanks': 800, # rate at which feed_tank is filled with water
    'Feed_EMU': 300,  #
    'Output_EMU': 330,  # More by CO2
    'Feed_LAROX': 7000,  # Spec of pump
    'Output_LAROX': 6585,  # Less due to solids, later depending on settings
    'Flush_LAROX': 7000,  # input FiltrateTank
    'Wash_LAROX': 7000,  #
    'Cloth_LAROX': 7000  #
}
for key in rates:
    rates[key] /= 60  # Convert to liters per minute


# Define durations in minutes for different events
durations = {
    'Fill_Larox': 2,  # later linked to density and cake size
    'Flush_Larox': 2,  # Static
    'Wash_Larox': 2,  # later linked wash settings
    'Cloth_Larox': 2,  # later linked to pressure settings
    'Cake_Larox': 2,  # Static
}

# Initialize the dynamic properties (start all tanks at 20% capacity) -> turn into function that updates all tanks.
tanks['Level_Liters'] = tanks['MaxCapacity'] * 0.2
tanks['Level_Percent'] = (tanks['Level_Liters'] / tanks['MaxCapacity'])*100
tanks['UsableCapacity_liters'] = tanks['HCapacity'] - tanks['Level_Liters']

print(tanks)
print(durations)
print(rates)


# Initialize the event queue
event_queue = []

# Function to schedule events
def schedule_event(event_duration, event_type, tank_name, rate):
    print(event_queue)
    heapq.heappush(event_queue, (event_duration, event_type, tank_name, rate))
    print(event_queue)



# Initial events for filling the feed tanks
for tank in ['FeedTank1']:
    available_capacity = tanks.loc[tanks['Tank'] == tank, 'UsableCapacity_liters'].values[0]
    feed_rate = rates['Feed_Tanks']
    fill_duration = available_capacity/ feed_rate
    schedule_event(fill_duration, 'stop_filling', tank, feed_rate)


# Function to process events
def process_event(event):
    event_duration, event_type, tank_name, rate = event
    print(f"Processing event: {event_type} for {tank_name} at time {event_duration}")

    if event_type == 'stop_filling':
        # Update the tank level
        tanks.loc[tanks['Tank'] == tank_name, 'Level_Liters'] += rate * event_duration
        tanks['Level_Percent'] = (tanks['Level_Liters'] / tanks['MaxCapacity'])*100
        tanks['UsableCapacity_liters'] = tanks['HCapacity'] - tanks['Level_Liters']

# Simulation loop
log = []

# Initialize start time
current_time = datetime(2024, 10, 1, 6, 0)

# Add 120 minutes (2 hours) to the current time
# new_time = current_time + timedelta(minutes=120)

# Print the new time in the desired format
# formatted_new_time = new_time.strftime("%a-%d-%m %H:%M")
# print(f"New time: {formatted_new_time}")



while event_queue:
    # Get the next event
    next_event = heapq.heappop(event_queue)
    event_duration = next_event[0]

    # Advance time to the event
    current_time += timedelta(minutes = event_duration)

    # Process the event
    process_event(next_event)

    # Log the state after processing the event
    log.append(tanks.copy())

# Convert the log to a DataFrame for analysis
#log_df = pd.concat(log, keys=[f"Step {i}" for i in range(len(log))], names=['Simulation Step'])
#print(log_df)




           Tank  MaxCapacity   HH    H    L   LL  HCapacity  LCapacity  \
0     FeedTank1         6500  0.9  0.8  0.2  0.1     5200.0     1300.0   
1     FeedTank2         6500  0.9  0.8  0.2  0.1     5200.0     1300.0   
2    SlurryTank         6500  0.9  0.8  0.2  0.1     5200.0     1300.0   
3  FiltrateTank         6500  0.9  0.8  0.2  0.1     5200.0     1300.0   

   Level_Liters  Level_Percent  UsableCapacity_liters  
0        1300.0           20.0                 3900.0  
1        1300.0           20.0                 3900.0  
2        1300.0           20.0                 3900.0  
3        1300.0           20.0                 3900.0  
{'Fill_Larox': 2, 'Flush_Larox': 2, 'Wash_Larox': 2, 'Cloth_Larox': 2, 'Cake_Larox': 2}
{'Feed_Tanks': 13.333333333333334, 'Feed_EMU': 5.0, 'Output_EMU': 5.5, 'Feed_LAROX': 116.66666666666667, 'Output_LAROX': 109.75, 'Flush_LAROX': 116.66666666666667, 'Wash_LAROX': 116.66666666666667, 'Cloth_LAROX': 116.66666666666667}
[]
[(292.5, 'stop_filling', 